<a href="https://colab.research.google.com/github/kinza7867/EventPalApp-project/blob/main/EmotionAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ======================================================
# STEP 1: Install Dependencies
# ======================================================
!pip install transformers torch matplotlib pandas ipywidgets nltk seaborn --quiet

# ======================================================
# STEP 2: Imports & Downloads
# ======================================================
import torch
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from transformers import pipeline
from nltk.tokenize import sent_tokenize
import seaborn as sns
import random

nltk.download('punkt')
nltk.download('punkt_tab')

# Set seaborn style for more attractive plots
sns.set(style="whitegrid", palette="pastel")

# ======================================================
# STEP 3: Load Emotion Detection Model
# ======================================================
emotion_analyzer = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=True
)

# ======================================================
# STEP 4: Emotion Intensity Helper
# ======================================================
def emotion_intensity(score):
    if score >= 0.6:
        return "High"
    elif score >= 0.35:
        return "Moderate"
    else:
        return "Low"

# ======================================================
# STEP 5: Insight Generator (Condensed for Less "Theory")
# ======================================================
def generate_insights(emotions):
    insights = ["<b>🧠 Quick Mental Health Insights:</b>"]

    # Condensed General Introduction
    insights.append("Your emotions are valid! Here's a snapshot with quick tips to manage or boost them. For more, expand below.")

    # Sadness (Condensed)
    if emotions.get("sadness", 0) > 0.25:
        insights.append("<br>• <b>Sadness Detected</b>: Possible distress.")
        insights.append("  → <i>Quick Tips:</i> Talk to a friend, take a walk, journal thoughts. Call 988 if needed.")

    # Fear/Anxiety (Condensed)
    if emotions.get("fear", 0) > 0.25:
        insights.append("<br>• <b>Anxiety Signals</b>: Worry detected.")
        insights.append("  → <i>Quick Tips:</i> Try 4-7-8 breathing, grounding exercises, break tasks small. Apps: Calm/Headspace.")

    # Anger (Condensed)
    if emotions.get("anger", 0) > 0.25:
        insights.append("<br>• <b>Frustration Present</b>: Tension noted.")
        insights.append("  → <i>Quick Tips:</i> Take a timeout, use 'I' statements, exercise to release energy. APA resources for more.")

    # Joy (Condensed)
    if emotions.get("joy", 0) > 0.25:
        insights.append("<br>• <b>Positive Vibes</b>: Joy spotted!")
        insights.append("  → <i>Quick Tips:</i> Share it, keep a gratitude journal, plan fun activities. Sustain with self-care.")

    # Love (Condensed)
    if emotions.get("love", 0) > 0.25:
        insights.append("<br>• <b>Connection Felt</b>: Love detected.")
        insights.append("  → <i>Quick Tips:</i> Express it, spend quality time, practice kindness. Nurture with communication.")

    # Surprise (Condensed)
    if emotions.get("surprise", 0) > 0.25:
        insights.append("<br>• <b>Surprise Elements</b>: Unexpected vibes.")
        insights.append("  → <i>Quick Tips:</i> Reflect and adapt, journal lessons, build flexibility.")

    # Quick General Tips
    insights.append("<br><b>🌟 Fast Well-Being Boosts:</b>")
    insights.append("• Hydrate & move 30 mins daily • Sleep well • Connect with others • Meditate 5 mins • Seek pro help if overwhelmed.")

    insights.append("<br><b>⚠ Note:</b> For fun & education only—not medical advice. Consult pros for real support.")
    return insights

# ======================================================
# Unique Feature: Fun Affirmation Generator
# ======================================================
def generate_affirmation(dominant_emotion):
    affirmations = {
        "joy": ["You radiate positivity—keep shining!", "Embrace the joy; it's your superpower!"],
        "sadness": ["This too shall pass; you're stronger than you know.", "Allow yourself grace; healing starts with kindness."],
        "anger": ["Breathe in calm, exhale tension.", "Your voice matters—channel it wisely."],
        "fear": ["Courage isn't absence of fear; it's action despite it.", "You're capable—take one step at a time."],
        "love": ["Love multiplies when shared—keep giving!", "You are worthy of all the love you give."],
        "surprise": ["Embrace the unexpected; adventure awaits!", "Surprises bring growth—stay open."]
    }
    return random.choice(affirmations.get(dominant_emotion, ["You're amazing—keep going!"]))

# ======================================================
# STEP 6: MULTI-SENTENCE EMOTION ANALYSIS
# ======================================================
def analyze_emotions(text):
    if not text.strip():
        return {}, [], ["No text provided"]

    sentences = sent_tokenize(text)
    emotion_totals = {}
    sentence_results = []

    for sent in sentences:
        results = emotion_analyzer(sent)[0]
        dominant = max(results, key=lambda x: x['score'])
        sentence_results.append((sent, dominant))

        for r in results:
            emotion_totals[r['label']] = emotion_totals.get(r['label'], 0) + r['score']

    for emotion in emotion_totals:
        emotion_totals[emotion] /= len(sentences)

    insights = generate_insights(emotion_totals)
    return emotion_totals, sentence_results, insights

# ======================================================
# STEP 7: Visualization (Reverted to Bar Chart with Enhanced Styling)
# ======================================================
def plot_emotions(emotions):
    colors = {
        "joy": "green",
        "sadness": "blue",
        "anger": "red",
        "fear": "orange",
        "love": "purple",
        "surprise": "gold"
    }

    labels = list(emotions.keys())
    scores = list(emotions.values())
    bar_colors = [colors.get(e, "gray") for e in labels]

    plt.figure(figsize=(10,5))
    bars = plt.bar(labels, scores, color=bar_colors, edgecolor='black', linewidth=1.2)
    plt.title("Overall Emotional Profile (Paragraph Level)", fontsize=14, fontweight='bold')
    plt.ylabel("Average Confidence Score", fontsize=12)
    plt.ylim(0,1)
    plt.grid(axis='y', alpha=0.3)
    plt.xticks(fontsize=10, rotation=45)
    plt.yticks(fontsize=10)

    # Add value labels on bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f"{yval:.2f}", ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

# ======================================================
# STEP 8: User Interface (Enhanced with Examples and Fun Elements)
# ======================================================
# Welcome Message for Attraction
display(HTML("<h1 style='text-align: center; color: #4CAF50;'>🌟 Discover Your Emotional World! 🌟</h1>"))
display(HTML("<p style='text-align: center; font-size: 16px;'>Unlock AI-powered insights into your feelings. Get fun affirmations, quick tips, and visualize your emotions. Test with examples below or share your own story!</p>"))

text_input = widgets.Textarea(
    placeholder="Describe your feelings here... What's on your mind today?",
    layout=widgets.Layout(width='100%', height='120px', border='1px solid #4CAF50', padding='10px', background_color='#f9f9f9')
)

button = widgets.Button(
    description="Analyze My Emotions!",
    button_style='success',
    layout=widgets.Layout(width='auto', margin='10px 0')
)

output = widgets.Output()

# Example Texts for Attraction
examples = {
    "Happy Day": "Today was amazing! I spent time with friends, laughed a lot, and felt so connected.",
    "Stressful Moment": "I'm overwhelmed with work deadlines and feel anxious about failing.",
    "Mixed Feelings": "I'm excited about the new job but sad to leave my old team."
}

def load_example(example_text):
    text_input.value = example_text
    on_click(None)  # Auto-analyze

example_buttons = []
for label, text in examples.items():
    btn = widgets.Button(description=label, button_style='info', layout=widgets.Layout(margin='5px'))
    btn.on_click(lambda b, t=text: load_example(t))
    example_buttons.append(btn)

example_box = widgets.HBox(example_buttons, layout=widgets.Layout(justify_content='center', margin='10px 0'))

def on_click(b):
    with output:
        output.clear_output()

        emotions, sentences, insights = analyze_emotions(text_input.value)

        # Sentence-Level with Styling
        display(HTML("<h3 style='color: #2196F3;'>📌 Dive into Sentences:</h3>"))
        for s, e in sentences:
            display(HTML(f"<p style='background-color: #f0f0f0; padding: 8px; border-radius: 5px;'>• <i>{s}</i><br>  → <b style='color: #FF5722;'>{e['label'].capitalize()}</b> ({e['score']:.2f})</p>"))

        # Overall Scores in a Table for Appeal
        display(HTML("<h3 style='color: #2196F3;'>📊 Emotion Breakdown:</h3>"))
        score_html = "<table style='width:100%; border-collapse: collapse;'>"
        score_html += "<tr><th>Emotion</th><th>Score</th><th>Intensity</th></tr>"
        for k, v in emotions.items():
            intensity = emotion_intensity(v)
            color = 'green' if intensity == 'Low' else 'orange' if intensity == 'Moderate' else 'red'
            score_html += f"<tr><td>{k.capitalize()}</td><td>{v:.2f}</td><td><span style='color:{color};'>{intensity}</span></td></tr>"
        score_html += "</table>"
        display(HTML(score_html))

        dominant_emotion = max(emotions, key=emotions.get)
        display(HTML(f"<h3 style='color: #2196F3;'>⭐ Your Main Vibe: <span style='color: #FF5722;'>{dominant_emotion.upper()}</span></h3>"))

        if emotions.get("sadness", 0) > 0.4 and emotions.get("fear", 0) > 0.3:
            display(HTML("<p style='color: red; background-color: #FFEBEE; padding: 10px; border-radius: 5px;'><b>🚨 Heads Up:</b> High distress vibes. Reach out for support— you're not alone!</p>"))

        # Plot (Bar Chart)
        plot_emotions(emotions)

        # Fun Affirmation (Unique Attraction)
        affirmation = generate_affirmation(dominant_emotion)
        display(HTML(f"<div style='background-color: #E8F5E9; padding: 15px; border-radius: 10px; text-align: center; margin: 10px 0;'><b>💫 Your Personal Affirmation:</b><br><i style='font-size: 18px; color: #4CAF50;'>{affirmation}</i></div>"))

        # Insights (Condensed, with Accordion for Details)
        acc = widgets.Accordion(children=[widgets.HTML("<br>".join(insights))])
        acc.set_title(0, 'Quick Tips & Insights (Click to Expand)')
        display(acc)

        # Call to Action
        display(HTML("<p style='text-align: center; font-style: italic; color: #757575;'>Loved this? Try another story or share with friends! What emotions will you discover next? 😊</p>"))

button.on_click(on_click)

display(widgets.VBox([example_box, text_input, button, output]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cpu
